In [1]:
import geopandas as gpd
import networkx as nx

# Load the GeoJSON file to inspect its structure
file_path = 'verkehrslinie-Garching-Basemap.geojson'
geo_data = gpd.read_file(file_path)

# Display the first few rows of the data to understand its structure
geo_data.head()

,fid,id,land,objektart,klasse,klasse_org,name,name_kurz,zweitname,nummer,...,spurweite,spurweite_name,typ,typ_name,kennung,bauwerk,bauwerk_name,fahrbahnachse,ebene,geometry
0,15347,DENWAT011Vb00024,NW,Fahrbahnachse,Kreisstraße,1306,Concordiastraße,Concordiastr.,None,K19,...,None,None,None,None,0511900001690,None,None,1.0,0,"LINESTRING (6.84029 51.47767, 6.84049 51.47762..."
1,15348,DENWAT011Vb0002O,NW,Fahrbahnachse,Kreisstraße,1306,Concordiastraße,Concordiastr.,None,K19,...,None,None,None,None,0511900001690,None,None,1.0,0,"LINESTRING (6.84029 51.47767, 6.84038 51.47755..."
2,15838,DENWAT01D0004a01,NW,Fahrbahnachse,"Landesstraße, Staatsstraße",1305,Osterfelder Straße,Osterfelder Str.,None,L450,...,None,None,None,None,0511900006620,None,None,1.0,0,"LINESTRING (6.88181 51.48747, 6.88191 51.48753..."
3,15933,DENWAT01D0004a1z,NW,Fahrbahnachse,Gemeindestraße,1307,Centroallee,Centroa.,None,None,...,None,None,None,None,0511900001635,None,None,1.0,0,"LINESTRING (6.87509 51.48856, 6.87515 51.48854..."
4,16238,DENWAT01D0004a85,NW,Fahrbahnachse,Bundesstraße,1303,Mülheimer Straße,Mülh. Str.,None,B223,...,None,None,None,None,0511900006330,None,None,1.0,0,"LINESTRING (6.86382 51.47992, 6.86384 51.47999..."


In [4]:
# Extract necessary columns for building the network graph
streets_data = geo_data[['name', 'geometry']]

# Build a NetworkX graph from the GeoDataFrame
G = nx.Graph()

# Add edges to the graph based on street geometries
for _, row in streets_data.iterrows():
    street_name = row['name']
    street_kennung = row['kennung']
    geometry = row['geometry']

    # If the geometry is a LineString, extract coordinates
    if geometry.geom_type == 'LineString':
        coords = list(geometry.coords)
        # Add nodes and edges for each segment in the LineString
        for i in range(len(coords) - 1):
            start = coords[i]
            end = coords[i + 1]
            G.add_edge(start, end, street_name=street_name)
    # If the geometry is a MultiLineString, process each LineString
    elif geometry.geom_type == 'MultiLineString':
        for linestring in geometry:
            coords = list(linestring.coords)
            for i in range(len(coords) - 1):
                start = coords[i]
                end = coords[i + 1]
                G.add_edge(start, end, street_name=street_name)

# Create Street Key Table: Map street names to edge keys
street_key_table = []
for edge in G.edges(data=True):
    street_key_table.append({'street_name': edge[2]['street_name'], 'edge': (edge[0], edge[1])})

# Create Edge Table: Details of all edges
edge_table = []
for edge in G.edges(data=True):
    edge_table.append({'start': edge[0], 'end': edge[1], 'street_name': edge[2]['street_name']})

# Create Vertex Table: List of all nodes
vertex_table = [{'vertex': node, 'coordinates': node} for node in G.nodes()]

import pandas as pd

# Convert to DataFrames for better representation
street_key_df = pd.DataFrame(street_key_table)
edge_df = pd.DataFrame(edge_table)
vertex_df = pd.DataFrame(vertex_table)

street_key_df

,street_name,edge
0,Concordiastraße,"((6.8402915, 51.4776746), (6.8404867, 51.47761..."
1,Concordiastraße,"((6.8402915, 51.4776746), (6.8403832, 51.47755..."
2,Duisburger Straße,"((6.8402915, 51.4776746), (6.8416044, 51.47815..."
3,Duisburger Straße,"((6.8402915, 51.4776746), (6.8378975, 51.47679..."
4,Concordiastraße,"((6.8402915, 51.4776746), (6.8421342, 51.47651..."
...,...,...
26013,Vestische Straße,"((6.880298, 51.5028419), (6.8804907, 51.5027197))"
26014,Skagerrakstraße,"((6.8242065, 51.4976524), (6.8243321, 51.49727..."
26015,Skagerrakstraße,"((6.8243321, 51.4972713), (6.8244415, 51.496946))"
26016,Skagerrakstraße,"((6.8244415, 51.496946), (6.8244797, 51.4968325))"


In [ ]:
from shapely.geometry import Point, LineString
from geopy.distance import geodesic

# Function to calculate the geodesic length of an edge
def calculate_edge_length(start, end):
    return geodesic(start[::-1], end[::-1]).meters

# Add length calculation to the Edge Table
edge_df['length_m'] = [
    calculate_edge_length(edge[0], edge[1]) for edge in G.edges()
]

# Reorganize the Edge Table with the new column
edge_df = edge_df[['id', 'start', 'end', 'street_name', 'kennung', 'length_m']]



In [6]:
# Update the Street Key Table to use 'kennung' as the index
street_key_df['kennung'] = geo_data.set_index('geometry').reindex(street_key_df['street_name'])['kennung'].values
street_key_df = street_key_df[['kennung', 'street_name', 'edge']].set_index('kennung')

# Update the Edge Table to include 'kennung' and use 'id' as the index
edge_df['kennung'] = geo_data.set_index('geometry').reindex(edge_df['street_name'])['kennung'].values
edge_df['id'] = range(1, len(edge_df) + 1)  # Generate unique IDs for edges
edge_df = edge_df[['id', 'start', 'end', 'street_name', 'kennung']].set_index('id')

# Update the Vertex Table to include unique vertex IDs and a column for edge IDs
vertex_df['vertex_id'] = range(1, len(vertex_df) + 1)
vertex_df['edge_ids'] = [
    [edge_id for edge_id, edge in enumerate(G.edges(data=True), start=1)
     if vertex in (edge[0], edge[1])]
    for vertex in vertex_df['vertex']
]




KeyboardInterrupt



In [5]:
edge_df.head()

,start,end,street_name
0,"(6.8402915, 51.4776746)","(6.8404867, 51.4776153)",Concordiastraße
1,"(6.8402915, 51.4776746)","(6.8403832, 51.4775518)",Concordiastraße
2,"(6.8402915, 51.4776746)","(6.8416044, 51.4781524)",Duisburger Straße
3,"(6.8402915, 51.4776746)","(6.8378975, 51.4767974)",Duisburger Straße
4,"(6.8402915, 51.4776746)","(6.8421342, 51.4765154)",Concordiastraße
